<a target="_blank" href="https://colab.research.google.com/github/yasstake/rusty-bot/blob/main/manual/session.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Sessionクラス解説

Session クラスはAgentの各イベント(on_tick, on_clock, on_update)時に引数として送られてきます。

内部に現在時刻 `current_timestamp`を保持することによりバックテスト、フォーワードテスト、本番どれであっても同一処理のAgent(BOT)が動作するようになっています。

このSessionのインスタンスを利用してAgent内の取引ロジックを開発してください。
使えるプロパティ／メソッドについて説明します。

このNoteの中では説明のためにSessionオブジェクトのインスタンス化や内部関数も使いますが、
ユーザがはSessionオブジェクトを自分で生成することはありません。

## プロパティ

### 現在時刻を表すプロパティ

* `Session.current_timestamp` 現在時刻をUnixtime(us)で保存取得します。

### 市況を示すプロパティ


### オーダーメソッド


### 自分のオーダー／ポジションを表すプロパティ




## 以下テストを兼ねたSessionオブジェクトの内部動作確認

In [ ]:
from rbot import BinanceMarket
from rbot import BinanceConfig
from rbot import MarketMessage
from rbot import Session
from rbot import Trade
from rbot import OrderSide
from rbot import ExecuteMode
from rbot import init_log

In [ ]:
from IPython.display import HTML
from json2html import json2html
import json

In [ ]:
config = BinanceConfig.TEST_BTCUSDT
market = BinanceMarket(config)

session = Session(market, execute_mode=ExecuteMode.Dummy, session_name="test_session", log_memory=True)

In [ ]:
HTML(json2html.convert(session.__repr__()))

In [ ]:
# 内部処理、バックテスト用にログを読み取らせると市況がUpdateされる。
from rbot import NOW
import rbot

trade = MarketMessage.from_trade(Trade(NOW(), OrderSide.Buy, 100, 1, "OrderId-1"))
session.on_message(trade)

HTML(json2html.convert(session.__str__()))
    

In [ ]:
session.account

In [ ]:
session.on_message(MarketMessage.from_trade(Trade(NOW(), OrderSide.Sell, 99, 1, "OrderId-2")))

HTML(json2html.convert(session.__str__()))

In [ ]:
# Agent用メソッド：　オーダーを発行すると,処理中オーダーにキュー(dummy_q)される
session.limit_order("Buy", 
                    100,    #価格
                    50,     # サイズ
)

HTML(json2html.convert(session.__str__()))

In [ ]:
session.dummy_q

In [ ]:
session.log_df

In [ ]:
# あたらしいメッセージがやってくるとQにあったオーダーが処理される。
# 買オーダー(100)よりも高い売り(101.0)がおこなわれてもオーダーには影響がない。
session.on_message(MarketMessage.from_trade(Trade(NOW(), OrderSide.Sell, 101, 50, "OrderId-2")))
HTML(json2html.convert(session.__str__()))

In [ ]:
session.dummy_q

In [ ]:
session.log_df

In [ ]:
# 買オーダー（１００）と同じ価格の売りがおこなわれているときも影響がない
# 本来は時間が経って板の最後までまわってきたら約定するが、辛めに判定
session.on_message(MarketMessage.from_trade(Trade(NOW(), OrderSide.Sell, 100.0, 50, "OrderId-3")))
HTML(json2html.convert(session.__str__()))

In [ ]:
# 買オーダー(100)よりも低い価格の売りがでてきたら、部分約定する（今回の例では、サイズ１が約定して49が残っている）
# ただし約定ログは分割せずremain_sizeをへらして全部約定のときにログを出す（あとで、まとめて約定したことにする）
session.on_message(MarketMessage.from_trade(Trade(NOW(), OrderSide.Sell, 99.9, 1, "OrderId-4")))
HTML(json2html.convert(session.__str__()))

# TODO: アカウント情報のUpdate

In [ ]:
session.log_df

In [ ]:
# のこり49の売りが発生すると全部約定する（注残リストからはなくなる）
session.on_message(MarketMessage.from_trade(Trade(NOW(), OrderSide.Sell, 99.9, 49, "OrderId-5")))
HTML(json2html.convert(session.__str__()))

In [ ]:
session.log_df

# まとめ

以上のような動きを約定ログ全部なめてバックテストを行なっています。

（ここまで細かくやる必要が果たしてあったのかは、これからのBot開発でわかるようになるはず）